### Objective

At a high level, these strategies all involve providing the model clear and specific instructions for what output it should produce. In this lab, you will:

- Define the output format & specify constraints
- Assign a persona or role
- Include examples
- Experiment with parameter values
- Utilize fallback responses
- Add contextual information
- Structure prompts with prefixes or tags
- Use system instructions
- Break down complex tasks
- Demonstrate the chain-of-thought
- Implement prompt iteration strategies to improve your prompts version by version

### Upgrade the Vertex AI SDK & Restart the Kernel

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

# note: Select Runtime > Restart Session > select Yes to restart the runtime.

In [ ]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

### Import packages

In [ ]:
PROJECT_ID = "qwiklabs-gcp-03-f2605c9303f1"  
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)